In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
import matplotlib.pyplot as plt
import tensorflow as tf # Used for Loading Datasets.

%matplotlib inline

/home/mohsin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mohsin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mohsin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mohsin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
(x_train, true), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train,x_test = x_train/255.0 ,x_test/255.0
inputs = x_train.reshape(x_train.shape[0],784)
test = x_test.reshape(x_test.shape[0],784)

In [3]:
test_true = pd.get_dummies(y_test).values
true = pd.get_dummies(true).values
print ("Inputs Shape {}".format(inputs.shape))
print ("True Shape {}".format(true.shape))

Inputs Shape (60000, 784)
True Shape (60000, 10)


In [4]:
def relu(x):
    return (x>0)*x

def relu2deriv(output):
    return output > 0

def sigmoid(x) : 
    return 1 / (1 + np.e**(-x))
def sigmoid_deriv(x): 
    return sigmoid(x) * (1 - sigmoid(x))

In [5]:
alpha = 0.2
hidden_size = 100

In [6]:
weights_0_1 = 2 * np.random.random((784,hidden_size)) - 1
weights_1_2 = 2 * np.random.random((hidden_size,10)) - 1

In [7]:
indices = list(range(len(inputs)))
for iteration in range(32):
    correct_pct = 0
    layer_2_error = 0
    np.random.shuffle(indices)
    inputs = inputs[indices]
    true = true[indices]
    for i in tqdm(range(len(inputs))):
        layer_0 = inputs[i:i+1]
        layer_1 = relu(np.dot(layer_0,weights_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        
        layer_2_error += np.sum((layer_2 - true[i:i+1]) ** 2)
        correct_pct += int(np.argmax(layer_2) == np.argmax(true[i:i+1]))
        
        layer_2_delta = (layer_2 - true[i:i+1]) * sigmoid_deriv(layer_2)
        layer_1_delta = np.dot(layer_2_delta,weights_1_2.T) * relu2deriv(layer_1)
        
        weights_1_2 -= alpha * np.dot(layer_1.T,layer_2_delta)
        weights_0_1 -= alpha * np.dot(layer_0.T,layer_1_delta)
        
        
    if iteration % 5 == 0:
        test_error = 0
        test_correct_pct = 0
        for test_iteration in range(len(x_test)):
            layer_0 = x_test[test_iteration:test_iteration+1].reshape(784)
            layer_1 = relu(np.dot(layer_0,weights_0_1))
            layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
            test_error += np.sum((test_true[test_iteration:test_iteration+1] - layer_2)**2)
            test_correct_pct += np.argmax(layer_2) == np.argmax(test_true[test_iteration: test_iteration+1])
        sys.stdout.write("\n" + "I:" + str(iteration) + " Test-Err:" + str(test_error/ float(len(x_train)))[0:5] + \
            " Test-Acc:" + str(test_correct_pct/ float(len(x_test)))[0:5]+ " Train-Err:" + str(layer_2_error/ float(len(x_train)))[0:5] +\
            " Train-Acc:" + str(correct_pct/ float(len(x_train)))[0:5])


  0%|          | 0/60000 [00:00<?, ?it/s]


I:0 Test-Err:0.045 Test-Acc:0.866 Train-Err:0.636 Train-Acc:0.705

  0%|          | 0/60000 [00:00<?, ?it/s]


I:5 Test-Err:0.032 Test-Acc:0.915 Train-Err:0.401 Train-Acc:0.833

 95%|█████████▌| 57012/60000 [00:32<00:01, 1761.14it/s]/home/mohsin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in power
  
  0%|          | 0/60000 [00:00<?, ?it/s]

...

  0%|          | 0/60000 [00:00<?, ?it/s]


I:25 Test-Err:0.028 Test-Acc:0.923 Train-Err:0.373 Train-Acc:0.854

  0%|          | 0/60000 [00:00<?, ?it/s]


I:30 Test-Err:0.035 Test-Acc:0.912 Train-Err:0.343 Train-Acc:0.865

100%|██████████| 60000/60000 [00:43<00:00, 1383.94it/s]


In [8]:
def predict_sample(layer_0):
    layer_1 = relu(np.dot(layer_0,weights_0_1))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    return np.argmax(layer_2)

In [9]:
sample_no = 123
print (f"Prediction: {predict_sample(x_test[sample_no].reshape(784))}")
print (f"True label: {np.argmax(test_true[sample_no])}")

Prediction: 6
True label: 6
